# Ejemplo de Knowledge Distillation
---

En este ejemplo vamos a ver como entrenar un modelo pequeño (número reducido capas y pesos) que replique el comportamiento de un modelo más grande que tiene un mayor accuracy pero con un tiempo de inferencia y uso de recursos mayor. Para ello, vamos a seguir un esquema teacher-student. De esta forma, esta técnica proporciona dos beneficios potenciales:


1.   Reducimos el tamaño del modelo por lo que ocupa menos en memoria y se ejecuta más rápido.
2.   Un modelo de tamaño de reducido con el rendimiento de uno mucho más complejo.

---

## 1. Instalar e importar las librerías necesarias

En este ejemplo vamos a trabajar con Pytorch y los modelos de torchvision

In [ ]:
!pip3 install torch torchvision torchinfo numpy

In [ ]:
from torchvision.models import resnet18, ResNet18_Weights, resnet50, ResNet50_Weights
from torchinfo import summary
import torch
import torchvision
import time
import numpy as np

## 2. Definir los modelos

Definimos el modelo teacher con unos pesos pre-entrenados. En este caso usamos ResNet50 pre-entrenada en ImageNet. Este modelo es muy conocido y ampliamente usado en clasificación de imágenes.

In [ ]:
teacher = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
preprocessing = ResNet50_Weights.IMAGENET1K_V1.transforms()
summary(teacher, input_size=(1, 3, 224, 224))

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 86.0MB/s]


Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 1000]                 --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 256, 56, 56]          --
│    └─Bottleneck: 2-1                   [1, 256, 56, 56]          --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           4,096
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]           --
│ 

Para el modelo student vamos a usar ResNet18, que es una versión reducida de ResNet50, por lo que vamos a intentar replicar el comportamiento de ResNet50 en ResNet18 que, a priori, obtiene peores resultados cuando se entrena desde cero.

In [ ]:
student = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
summary(student, input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 1000]                 --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]           --
│

## 3. Descargar la base de datos

Vamos a trabajar con un subconjunto de ImageNet (conjunto t3 de entrenamiento). Nos lo descargamos y descomprimimos.

In [ ]:
!wget https://www.ac.uma.es/~fcastro/files/imagenet.tar.gz
!tar -xzf imagenet.tar.gz

--2023-12-20 09:05:31--  https://www.ac.uma.es/~fcastro/files/imagenet.tar.gz
Resolving www.ac.uma.es (www.ac.uma.es)... 150.214.109.5
Connecting to www.ac.uma.es (www.ac.uma.es)|150.214.109.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 738370955 (704M) [application/x-gzip]
Saving to: ‘imagenet.tar.gz’

imagenet.tar.gz     100%[===================>] 704.17M  16.3MB/s    in 44s     

2023-12-20 09:06:16 (16.0 MB/s) - ‘imagenet.tar.gz’ saved [738370955/738370955]



## 4. Definir un data loader

Una vez descargados los datos, tenemos que crear un DataLoader de Pytorch para poder usarlos con nuestro modelo.


In [ ]:
dataset = torchvision.datasets.ImageFolder(root='./imagenet', transform=preprocessing)
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## 5. Entrenamiento del modelo student

Realizamos unas épocas para hacer que los pesos del modelo student repliquen el comportamiento del modelo teacher. Para ello, para cada batch, hacemos una inferencia del modelo teacher para obtener su comportamiento y luiego intentamos replicar la salida en el modelo student. Para ello, usamos la base de datos que hemos descargado en el punto 3. Además, como función de loss usamos la Hinton Loss cuya ecuación es la siguiente:

Loss = $-\sum_{c=1}^My_{o,c}\log(\frac{p_{o,c}}{T})$

Básicamente, es una Crossentropy loss con los logits divididos por la T (temperatura) cuyo valor normalmente es 2.0. Si usamos T=1.0, estaríamos usando una Crossentropy loss normal y corriente.

In [ ]:
n_epochs = 50
opt = torch.optim.Adam(student.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
teacher.eval()
student.train()
T = 2.0
for epoch in range(n_epochs): # Entrenamos n epocas
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    time_start = time.time()
    for inputs, labels in train_data_loader: # Obtenemos todos los batch de entrenamiento y los usamos para entrenar
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        opt.zero_grad()
        with torch.no_grad():
          outs_teacher = teacher(inputs) / T
          outs_teacher = torch.nn.functional.softmax(outs_teacher, dim=1)

        outs_student = student(inputs)
        loss = loss_fn(outs_student / T, outs_teacher)
        train_running_loss += loss.item()
        _, preds = torch.max(outs_student.data, 1)
        train_running_correct += (preds == labels).sum().item()
        counter = counter + 1
        loss.backward()
        opt.step()

    epoch_loss = train_running_loss / counter
    epoch_acc = 100. * (train_running_correct / len(train_data_loader.dataset))
    time_end = time.time() - time_start
    print(f'** Summary for epoch {epoch}: '
		f'loss: {epoch_loss:#.3g}, acc: {epoch_acc:#.3g}]  '
		f'time: {time_end:.3f}s **')

## 6. Exportar el modelo

Una vez que hemos realizado el entrenamiento modelo student, exportamos el modelo a un fichero para poder usarlo en nuestra aplicación.

In [ ]:
student.eval()
torch.save(student.state_dict(), '.student.pt')

Además, vamos a realizar una  inferencia de prueba para analizar el rendimiento del modelo teacher y del student.

In [ ]:
image = torch.Tensor(np.random.rand(1,3,224,244)).float().cuda()

# Teacher
time_start = time.time()
teacher(image)
time_end = time.time() - time_start
print(f'Execution time of the teacher model: {time_end:.3f}s')

# Student
time_start = time.time()
student(image)
time_end = time.time() - time_start
print(f'Execution time of the student model: {time_end:.3f}s')